## 1. Import Necessary Modules

In [1]:
import os
import time

%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
from altusi.utils.logger import *
import altusi.utils.visualizer as vis
from altusi.utils import misc

In [3]:
import numpy as np
import cv2 as cv

import mxnet as mx
import gluoncv

## 2. Load SSD Network

In [4]:
list(filter(lambda x: x.startswith('ssd'), gluoncv.model_zoo.get_model_list()))

['ssd_300_vgg16_atrous_voc',
 'ssd_300_vgg16_atrous_coco',
 'ssd_300_vgg16_atrous_custom',
 'ssd_512_vgg16_atrous_voc',
 'ssd_512_vgg16_atrous_coco',
 'ssd_512_vgg16_atrous_custom',
 'ssd_512_resnet18_v1_voc',
 'ssd_512_resnet18_v1_coco',
 'ssd_512_resnet50_v1_voc',
 'ssd_512_resnet50_v1_coco',
 'ssd_512_resnet50_v1_custom',
 'ssd_512_resnet101_v2_voc',
 'ssd_512_resnet152_v2_voc',
 'ssd_512_mobilenet1.0_voc',
 'ssd_512_mobilenet1.0_coco',
 'ssd_512_mobilenet1.0_custom',
 'ssd_300_vgg16_atrous_voc_int8',
 'ssd_512_mobilenet1.0_voc_int8',
 'ssd_512_resnet50_v1_voc_int8',
 'ssd_512_vgg16_atrous_voc_int8']

In [5]:
ctx = mx.context.gpu(0) if mx.context.num_gpus() else context.cpu()

LOG(INFO, 'Device in Use:', ctx)

[2019-08-28 10:24:22] [INFO]  Device in Use: gpu(0)


In [6]:
net = gluoncv.model_zoo.get_model('ssd_300_vgg16_atrous_voc', ctx=ctx)

net.load_parameters('ssd_300_vgg16_atrous_voc_best.params')

net.reset_class(classes=['person'], reuse_weights=['person'])

In [7]:
net.collect_params().reset_ctx(ctx)

In [8]:
L = list(net.collect_params().values())

In [9]:
X = mx.nd.random.uniform(shape=(1, 3, 300, 300), ctx=ctx)

cids, scores, bboxes = net(X)

## 3. Test with Images

In [10]:
def filter_bboxes(bboxes, scores, class_ids, thresh=0.5):
    ids = np.where(scores.asnumpy().reshape(-1) > thresh)[0]

    if len(ids):
        return bboxes[ids], scores[ids], class_ids[ids]
    else:
        return None, None, None


def ssd_predict(net, image, ctx, thresh=0.5, img_dim=300):
    x, img = gluoncv.data.transforms.presets.ssd.transform_test(mx.nd.array(image), short=img_dim)
    x = x.as_in_context(ctx)

    class_ids, scores, bboxes = net(x)

    if len(bboxes[0]) > 0:
        bboxes, scores, class_ids = filter_bboxes(bboxes[0], scores[0], class_ids[0], thresh)

        if bboxes is not None:
            classes = [net.classes[int(idx.asscalar())] for idx in class_ids]

    return class_ids, scores, bboxes, img

In [11]:
def rescale_bboxes(bboxes, dims, new_dims):
    H, W = dims
    _H, _W = new_dims
    
    _bboxes = []
    for bbox in bboxes:
        bbox = bbox.asnumpy()
        bbox = bbox / np.array([W, H, W, H]) * np.array([_W, _H, _W, _H])
        _bboxes.append(bbox)
        
    return _bboxes

In [71]:
image_path = '/home/brian/Pictures/annie000.jpeg'
image_path = '/home/brian/Pictures/annie001.jpeg'
image_path = '/home/brian/Pictures/human000.jpg'
image_path = '/home/brian/Pictures/coopmart000.png'
image_path = '/home/brian/Pictures/coopmart001.png'
image_path = '/home/brian/Pictures/coopmart002.png'
# image_path = '/home/brian/Pictures/face-015.jpg'
# image_path = '/home/brian/Pictures/rdf010.png'
# image_path = '/home/brian/Pictures/sanha000.png'

image_fname, ext = misc.get_fname(image_path)
image = cv.imread(image_path)

image.shape

(1080, 1875, 3)

In [72]:
cids, scores, bboxes, img = ssd_predict(net, image, ctx, thresh=0.5)

In [73]:
rs_bboxes = rescale_bboxes(bboxes, img.shape[:2], image.shape[:2])

In [74]:
out_image = vis.drawObjects(image, rs_bboxes, 
                            color=vis.COLOR_RED_LIGHT,
                           thickness=4)

In [75]:
cv.imwrite(f'images/{image_fname}{ext}', out_image)

True